#|hide notes

## Developer notes

This is a first attempt at using nbdev to develop a package, commentary is added to the code to explain the process. [nbdev](https://github.com/fastai/nbdev) is a tool to develop packages using python jupyter notebooks. It provides a set of tools (can check via `nbdev_help`) which makes what you write in this notebook a package. If you follow conventions it uses it also includes testing which can be automatically turned into github actions, and releases for both conda and pypi. It accomplishes this by transforming notebook code into a standard python package and using information in certain files and folders (notably `settings.ini` and `*.ipynb`) to create a package.

I think it's important to organize other files you add to this structure so you're aware of which files are auto generated and which ones aren't. This also encourages a different way to approach the code that is focused on explanation more so than pure structure. For examply when creating the class is this notebook I purposefully didn't put many methods with the intiail declaration. Instead I included the basics and then used `fastcore.utils.patch` to "patch" in additional methods. These methods are then associated with the class (I believe it uses the typeclass of self to know where it goes) when converted. In terms of final code it looks a little less organized but in terms of process you can create your object, add some tests to ensure it works, then patch in a function, test that inidividual function, and continue till all functions are added. This goes better with explanations but worse for at a glance results. Because of this library imports are not all at the top of the notebook. Instead they are added as needed as the code is written for understanding. This may cause some issues with remembering to include packages in the `conda.env.yml`.

A plus from this is docs are also auto created. This means each section should be demarked by a `#|hide` or `#|export` tag, with the `#|export` tag being used for all code that makes it to production. It appears `#|hide` can also be applied to markdown to exclude it, having `#|export` in markdown doesn't force include it though, so more likely an interaction with any tags. If you use neither it appears in the docs but not in the production code. I should also mention quickly that it appears `nbdev` is still relatively new but it feels like you should be able to make any kind of python package with this, including CLI. Also note that `#|export` which needed for including to modules does not include in documentation always, this looks like it applies to all non-function exports. All code blocks seen are either from functions or are not exported.

I believe the tests that go to github actions are ones using the `fastcore.test` library. So using this should offer that functionality.

As a more minor note I've developed in [VSCode](https://code.visualstudio.com/) using a jupyter notebook environment built from `conda.env.yml` and pointing my Jupyter Server to the associated `python` created (`./venv/bin/python`). I find this makes most things easier but some integration (certain auto-completions) of `nbdev` is currently designed to work through a server based [jupyter notebook](https://jupyter.org/).

Other sections of markdown labeled with `#|hide` should be assumed as further developer notes and not intended for the final product.

#| hide

## References for development

1. [nbdev_cards](https://github.com/fastai/nbdev_cards), an example they use in their [video tutorial](https://www.youtube.com/watch?v=l7zS8Ld4_iA)

#|hide

This bit of code indicates which python file (in this case `pingme` it should create within the package. Each notebook should export to a different file).

In [ ]:
#| default_exp core

#|hide

The following folders/files are included outside of the nbdev framework, I don't want to modify nbdev autogenerated files of course.
- `conda.env.yml` - conda environment file for development
- `./config/`
    - `config.env` - environment variables
- `./output` - folder for output files
- `./cards`
    - `./cards/default_card.json` - default card for the pingme app

And within the framework I've modified
- `*.ipynb` - notebooks, the prepending ##_ indicates the order they should be run in, but does not affect package naming
    - `index.ipynb` is special as it generates the `README.md` file
- `settings.ini` - configuration file for nbdev

Other files were autogenerated by nbdev and the majority are not intended to be modified.


# PingMe

To install notebook development environment run the following command:
`conda create env -p ./.venv --file conda.env.yml`

NOTE: this may not be the nbdev approach but leaving for now, pretty sure I use pip for envs.

#|hide

I thought of `#|hide` as dev mode and `#|export` as production. This means that if you don't `#|export` what you need it may not work and that you have to identify what is for development only and `#hide` it. This can cause some issues as libraries in `#|hide` may be used in dev and not work in production. This is just something to be aware of. The point of also keeping it in dev mode is to minimize the bloat for production.

In [ ]:
#|hide
# Development libraries, not required for package.
from nbdev.showdoc import *
import fastcore.test # For running tests

In [ ]:
#|export
# Inlcuded libraries, other libraries are included with the methods that use them.
# FIXME: Currently this is not showing in the docs, currently it seems no librs are
from fastcore.utils import patch # decorator to patch in new methods to a class

#|hide

I'm doing project based development for paths. I have an expectation that all paths I use in the program should be relative to the current project folder and if not that those values are provided via a config or args. No information about the system paths should be available and thus the code base should also be safe for public repositories as well. We don't want to expose system structure. To handle this the variables used are set in a environmental variable that points to a file which is in `./gitignore`. All env variables for this notebook are prepended with the program name `PINGME_`. This is to avoid potential conflicts with other programs. I assume a default `config.env` when looking though automatically.

In [ ]:
#|hide
# For development sets real config values which are not stored in git
# Check if config.env is present
import os
from pathlib import Path # for type hinting and file checking

if os.environ.get("PINGME_CONFIG_PATH") is None and Path("./config/config.env").is_file():
    os.environ["PINGME_CONFIG_PATH"] = "./config/config.env"
PINGME_CONFIG_PATH = os.environ["PINGME_CONFIG_PATH"]

## Config Variables
Used for running the notebook in dev mode, not used in sections with `#|export`

#|hide
I load all variables which are provided via into a config dict currently. These are loaded from default values, .env file, and environmental variables. .env and environmental variables do not need to include all values and will overwrite default values if the key is included. You can see all the variable names in the Notebook Variables section.

In [ ]:
#|export
from dotenv import dotenv_values # for loading config from .env files
import os

def get_config(config_path) -> dict:
    # Order of precedence: environment variables > .env file > default values
    return {
        **dotenv_values("./config/config.default.env"),       # load global default vars
        **dotenv_values(config_path),            # load specific vars, path of config is stored in ENV variable DEMUX_ILLUM_CONFIG_PATH
        **os.environ,                                         # override loaded values with ENV variables
        'PROJECT_PATH': os.getcwd()                           # set the project path relative to notebook
    }

## Notebook Variables

Here you can see included potential variables and their ENV variable names.

#|hide

As these variables are only used within the context of this notebook for testing they are **not** exported. For convention I gather them all from `get_config(PATH)` and name all variables in ALL_CAPS. This is to make it clear that these are not variables that are used in the code base. I also use the `#|hide` tag to make sure they are not exported.

In [ ]:
import distutils.util # for converting strings to bools

config = get_config(PINGME_CONFIG_PATH)

WEBHOOK_URL = config['PINGME_WEBHOOK_URL']
EMAIL_FROM = config['PINGME_EMAIL_FROM']
EMAIL_TO = config['PINGME_EMAIL_TO']
SMTP_HOST = config['PINGME_SMTP_HOST']
SMTP_PORT = config['PINGME_SMTP_PORT']
SMTP_USER = config['PINGME_SMTP_USER']
SMTP_PASSWORD = config['PINGME_SMTP_PASSWORD']
LOG_FILE = config['PINGME_LOG_FILE']
TITLE = config['PINGME_TITLE']
TEXT = config['PINGME_TEXT']
PAYLOAD_FILE = config['PINGME_PAYLOAD_FILE']
PAYLOAD_CONTEXT = config['PINGME_PAYLOAD_CONTEXT']
SEND_EMAIL = distutils.util.strtobool(config['PINGME_SEND_EMAIL'])
SEND_WEBHOOK = distutils.util.strtobool(config['PINGME_SEND_WEBHOOK'])
SEND_LOG_FILE = distutils.util.strtobool(config['PINGME_SEND_LOG_FILE'])

#|hide

Just checking env variables are as expected. Do not include output in repo as it'll contain system specific potentially sensative information.

In [ ]:
#|hide output
print(config)

## PingMe class
> Is used to send notifications to a webhook (designed for use with [incoming webhooks](https://docs.microsoft.com/en-us/microsoftteams/platform/webhooks-and-connectors/how-to/add-incoming-webhook in Microsoft Teams)), e-mail, and/or a log file. The class stores a json payload which can have variables for replacement denoted by `${key_name}`. A payload_context is passed to replace all variables in the payload. As the most common use will be to send a message with a Title/Subject and Text/Body these can be optionally passed and will work with the `default_card.json` template. For card info and designs see [here](https://adaptivecards.io/samples/).

#|hide

NOTE: Functions are added to the class using the `@fastcore.utils.patch` decorator with typehint `self:PingMe` to ensure it's added to the class. Also note it uses the comments on the per line of init to generate the docs, this is a feature from nbdev.

In [ ]:
#|export
import json # to manage json payloads
from pathlib import Path # for type hinting and file checking

class PingMe:
    """
    PingMe class which notifies via either a webhook or email
    """
    def __init__(self,
                 title:str = None, # Title of email, used in default card
                 text:str = None,  # Text of email, used in default card
                 payload_file:Path = "./cards/default_card.json", # Payload file to send to webhook
                 payload_context:dict = {}, # Variables to substitute in payload
                 ):
        self.title, self.text, self.payload_file, self.payload_context = title, text, payload_file, payload_context
        self.payload:json = None # Stores the unresolved payload contents

        #Check if payload_file is a valid path
        if not Path(self.payload_file).is_file():
            raise ValueError("Payload file does not exist")

        with open(self.payload_file, 'r+') as f:
            content = f.read()
            self.payload = json.loads(content)

        if self.title is not None:
            self.payload_context["title"] = self.title

        if self.text is not None:
            self.payload_context["text"] = self.text
    def __str__(self):
        return (
        f"""PingMe object with:
    title: {self.title}
    text: {self.text}
    payload_file: {self.payload_file}
    payload_context: {self.payload_context}
    payload:
{json.dumps(self.payload, indent=4, sort_keys=True)}"""
     )
    def __repr__(self):
        return self.__str__()

Check the notification object loading with defaults and the contents of it

In [ ]:
notification = PingMe(title=TITLE, text=TEXT)
notification

Functionality to resolve the payload, which means changing the variables and returning it as a `str` instead of a `json` object.

In [ ]:
#|export
import re # regular expression for parsing

@patch
def resolved_payload(self:PingMe) -> str:
    """
    Resolves the payload by substituting variables in the `self.payload` with values from the `self.payload_context` and ensures all variables are accounted for
    """
    if self.payload is None:
        # Ensure there is a payload
        raise ValueError("Payload is None")
    str_temp = json.dumps(self.payload) # convert payload to string
    for key in self.payload_context:
        # Substitute all variables in payload with values from payload_context, it can also be set up that their are no variables in the payload
        str_temp = str_temp.replace("${"+key+"}", self.payload_context[key])
    if re.search("\${.*}", str_temp):
        # Check if there are any variables left, this is not allowed
        raise ValueError("Unresolved variables in payload")
    return str_temp

In [ ]:
notification.resolved_payload()

Functionality to send the resolved_payload to a webhook

In [ ]:
#|export
import requests # to send requests to webhooks

@patch
def send_to_webhook(self:PingMe, url:str) -> dict:
    """
    Send the payload to a provided webhook `url` and returns response info when possible
    """
    webhook_header = {'Content-Type': 'application/json'}
    if url is None:
        raise Exception("Webhook URL not set")
    # Send message to webhook
    try:
        response = requests.post(url, data=self.resolved_payload(), headers=webhook_header)
    except Exception as e:
        raise Exception(f"Error sending message to webhook: {e}")
    return {"status_code": response.status_code, "response": response.text}

Basic tests

In [ ]:
# Test it fails with a bad url
with fastcore.test.ExceptionExpected(): notification.send_to_webhook("https://badhost")

If there's an error with this please check if you have a valid `WEBHOOK_URL` in the config.

In [ ]:
if SEND_WEBHOOK:
    notification.send_to_webhook(WEBHOOK_URL)

Functionality to send the resolved_payload to an e-mail address

In [ ]:
#|export
import smtplib # to send emails
import email.mime.text # to format emails

@patch
def send_to_email(self:PingMe, email_from:str, email_to:str, smtp_host:str, smtp_port:int=25, smtp_username=None, smtp_password=None) -> bool:
    email_status = False
    html_content = f"""
    <html>
    <head>
        <meta http-equiv="Content-Type" content="text/html; charset=us-ascii">
        <script type="application/adaptivecard+json">
            {self.resolved_payload()}
        </script>
    </head>
        <body>
            This is a sample body
        </body>
    </html>
    """
    msg = email.mime.text.MIMEText(html_content, 'html')
    msg['Subject'] = self.title
    msg['From'] = email_from
    msg['To'] = email_to
    email_connection = smtplib.SMTP(smtp_host, smtp_port)
    try:
        email_connection.ehlo()
        email_connection.starttls()
        email_connection.ehlo()
        email_connection.login(smtp_username, smtp_password)
        email_connection.sendmail(email_from, email_to, msg.as_string())
        email_status = True
    finally:
        email_connection.quit()
        return email_status


If there's error's here please check the type of error. ConnectionRefusedError indicates there's issues with connecting to the SMTP email server.

In [ ]:
if SEND_EMAIL:
    notification.send_to_email(EMAIL_TO, EMAIL_FROM, SMTP_HOST, SMTP_PORT, SMTP_USER, SMTP_PASSWORD)

Basic test

In [ ]:
# Test if not provided a proper smtp host
with fastcore.test.ExceptionExpected(): notification.send_to_email(EMAIL_FROM, EMAIL_TO, "badhost")

!|hide
The `send_to_logfile` function right now only sends a title and text, along with a timestamp. This is based off the default card. Some better solution should be found for this in the future. The goal is to not send raw json to the log file as it wont be readable. Perhaps storing each file in a folder and then logging the date and link to file?

In [ ]:
#|export
import datetime # to get current date and time which is used in logging

@patch
def send_to_log_file(self:PingMe, log_file:str) -> bool:
    """
    Send message to log_file
    TODO: The log file only lods title and text right now (not payload), while payload can be included it is not good for parsing. Need to think of a solution for this. Could be to save each payload as a seperate file and the log is a list of files.
    """
    if log_file is None:
        raise Exception("Log file not set")
    with open(log_file, "a") as f:
        # Write the current time
        f.write(f"{datetime.datetime.now()}\t{self.title}\t{self.text}\n")
        f.write("\n")
    return True

In [ ]:
if SEND_LOG_FILE:
    notification.send_to_log_file(LOG_FILE)

In [ ]:
# Test if not provided a proper log file
with fastcore.test.ExceptionExpected(): notification.send_to_log_file(None)

In [ ]:
#|export
# Make a CLI function using `call_parse` to handle arguments
from sys import stderr
from fastcore.script import call_parse
import distutils.util # to convert string to bool
import sys
# Ensure settings.ini contains `console_scripts = pingme=pingme.pingme:cli`, this makes the call as `pingme` and calls the cli function found in package pingme.pingme
@call_parse
def cli(
    # webhook_url:str = None, # URL of the teams/slack webhook
    # email_from:str = None, # Email address to send from
    # email_to:str = None, # Email address to send to
    # smtp_host:str = None, # SMTP host to use
    # smtp_port:int = None, # SMTP port to use
    # smtp_user:str = None, # SMTP username to use
    # smtp_password:str = None, # SMTP password to use
    # log_file:str = "./output/log.txt", # Log file to use
    # title:str = None, # Title of the message
    # text:str = None, # Text of the message
    # payload_file:str = "./cards/default_card.json", # Path to a file containing the payload
    # payload_context:str = None, # Context to use for the payload (variable replacement)
    # webhook:bool = False, # Send to webhook
    # email:bool = False, # Send to email
    # log:bool = False, # Send to log
    config_file:str # Path to a config file
    ):
    """
    The Command Line Interface (CLI) for the pingme package. This is the main entry point for the package. Currently only allows for a config file to be provided, but can be extended to allow for args. Command line passing is handled with @call_parse decorator.
    """
    config = get_config(config_file)

    WEBHOOK_URL = config['PINGME_WEBHOOK_URL']
    EMAIL_FROM = config['PINGME_EMAIL_FROM']
    EMAIL_TO = config['PINGME_EMAIL_TO']
    SMTP_HOST = config['PINGME_SMTP_HOST']
    SMTP_PORT = config['PINGME_SMTP_PORT']
    SMTP_USER = config['PINGME_SMTP_USER']
    SMTP_PASSWORD = config['PINGME_SMTP_PASSWORD']
    LOG_FILE = config['PINGME_LOG_FILE']
    TITLE = config['PINGME_TITLE']
    TEXT = config['PINGME_TEXT']
    PAYLOAD_FILE = config['PINGME_PAYLOAD_FILE']
    PAYLOAD_CONTEXT = config['PINGME_PAYLOAD_CONTEXT']
    SEND_EMAIL = distutils.util.strtobool(config['PINGME_SEND_EMAIL'])
    SEND_WEBHOOK = distutils.util.strtobool(config['PINGME_SEND_WEBHOOK'])
    SEND_LOG_FILE = distutils.util.strtobool(config['PINGME_SEND_LOG_FILE'])

    notification = PingMe(title=TITLE, text=TEXT)

    if SEND_WEBHOOK:
        notification.send_to_webhook(WEBHOOK_URL)
        print("Sent to webhook", file=sys.stderr)
    if SEND_EMAIL:
        notification.send_to_email(EMAIL_FROM, EMAIL_TO, SMTP_HOST, SMTP_PORT, SMTP_USER, SMTP_PASSWORD)
        print("Sent to email", file=sys.stderr)
    if SEND_LOG_FILE:
        notification.send_to_log_file(LOG_FILE)
        print("Sent to logfile", file=sys.stderr)

    return True

Test CLI by calling it as a function, should also export and run from command line to check as well.

In [ ]:
cli(PINGME_CONFIG_PATH)

#|hide

[Add in-notebook export cell](https://nbdev.fast.ai/tutorial.html#add-in-notebook-export-cell), which generates the .pymodules. Recommended to run `nbdev_prepare` to export, test and clean

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()